In [43]:
from lxml import html  
import requests
from time import sleep
import json
import argparse
from collections import OrderedDict
from time import sleep
import csv

def parse(ticker):
    url = "http://finance.yahoo.com/quote/%s?p=%s"%(ticker,ticker)
    response = requests.get(url, verify=False)
    print ("Parsing %s"%(url))
    sleep(4)
    parser = html.fromstring(response.text)
    quote = parser.xpath('//span[contains(@class,"Trsdu(0.3s) Fw(b)")]')[0].text
    quote_currency = parser.xpath('//div[contains(@class,"C($tertiaryColor) Fz(12px)")]//span')[0].text.split()[-1]
    summary_table = parser.xpath('//div[contains(@data-test,"summary-table")]//tr')
    summary_data = OrderedDict()
    other_details_json_link = "https://query2.finance.yahoo.com/v10/finance/quoteSummary/{0}?formatted=true&lang=en-US&region=US&modules=summaryProfile%2CfinancialData%2CrecommendationTrend%2CupgradeDowngradeHistory%2Cearnings%2CdefaultKeyStatistics%2CcalendarEvents&corsDomain=finance.yahoo.com".format(ticker)
    summary_json_response = requests.get(other_details_json_link)
    try:
        json_loaded_summary =  json.loads(summary_json_response.text)
        quoteSummary = json_loaded_summary["quoteSummary"]["result"][0]
        for each in quoteSummary:
             print ("============="+each+"================")
             print (quoteSummary[each])
        
        #price = quoteSummary["financialData"]["currentPrice"]['raw']
        financial_currency=quoteSummary["financialData"]["financialCurrency"]
        eps = quoteSummary["defaultKeyStatistics"]["trailingEps"]['raw']
        book_value = quoteSummary["defaultKeyStatistics"]["bookValue"]['raw']
        print(summary_data)
#         for table_data in summary_table:
#             raw_table_key = table_data.xpath('.//td[contains(@class,"C($primaryColor)")]//text()')
#             raw_table_value = table_data.xpath('.//td[contains(@class,"Ta(end)")]//text()')
#             table_key = ''.join(raw_table_key).strip()
#             table_value = ''.join(raw_table_value).strip()
#             summary_data.update({table_key:table_value})
        summary_data.update({'ticker':ticker,'url':url,'quote':quote,'eps':eps,'book_value':book_value,
                             'quote_currency':quote_currency,'financial_currency':financial_currency})
        return summary_data
    except:
        print ("Failed to parse json response")
        return {"error":"Failed to parse json response"}
        

#argparser = argparse.ArgumentParser()
#argparser.add_argument('ticker',help = '')
#args = argparser.parse_args()
# ticker = args.ticker
ticker ="D05.SI"
print ("Fetching data for %s"%(ticker))
scraped_data = parse(ticker)
print ("Writing data to output file")

#with open('%s-summary.json'%(ticker),'w') as fp:
     #json.dump(scraped_data,fp,indent = 4)
        

Fetching data for D05.SI


/usr/local/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'finance.yahoo.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


Parsing http://finance.yahoo.com/quote/D05.SI?p=D05.SI
=============recommendationTrend================
{'trend': [{'period': '0m', 'strongBuy': 0, 'buy': 0, 'hold': 0, 'sell': 0, 'strongSell': 0}, {'period': '-1m', 'strongBuy': 6, 'buy': 8, 'hold': 10, 'sell': 0, 'strongSell': 0}, {'period': '-2m', 'strongBuy': 6, 'buy': 7, 'hold': 10, 'sell': 1, 'strongSell': 0}, {'period': '-3m', 'strongBuy': 5, 'buy': 7, 'hold': 11, 'sell': 1, 'strongSell': 0}], 'maxAge': 86400}
=============earnings================
{'maxAge': 86400, 'earningsChart': {'quarterly': [{'date': '4Q2017', 'actual': {'raw': 0.48, 'fmt': '0.48'}, 'estimate': {'raw': 0.48, 'fmt': '0.48'}}, {'date': '1Q2018', 'actual': {'raw': 0.59, 'fmt': '0.59'}, 'estimate': {'raw': 0.56, 'fmt': '0.56'}}, {'date': '2Q2018', 'actual': {'raw': 0.54, 'fmt': '0.54'}, 'estimate': {'raw': 0.55, 'fmt': '0.55'}}, {'date': '3Q2018', 'actual': {'raw': 0.55, 'fmt': '0.55'}, 'estimate': {'raw': 0.55, 'fmt': '0.55'}}], 'currentQuarterEstimateDate': '4